Выполнила: Анастасия Плавина

## Тема: Введение, примеры задач, бизнес- и ML-метрики

**1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**
- Корреляция Спирмена реального и прогнозируемого рангов рекомендаций
- Fraction of Concordance Pairs - метрика того, насколько высока концентрация интересных товаров в начале списка рекомендаций
- Discounted cumulative gain at K — модификация cumulative gain at K, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции
- метрики качества ранжирования на основе каскадной модели PFound и Expected reciprocal rank (ERR)

In [1]:
import pandas as pd
import numpy as np

#### Задание 1

In [2]:
def hit_rate_at_k(recommended_list, bought_list, k):
    """calculating hit rate@k"""
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)

    return (flags.sum() > 0) * 1


In [3]:
recommended_list = [143, 156, 1134, 27, 1543, 3345, 533, 11, 43, 521] #id товаров
bought_list = [521, 32, 991, 27, 3345]

In [4]:
# посмотрим hit rate при разных k 
ks = [1, 3, 5, 10]
for k in ks:
    print(f'Hit rate @{k}: {hit_rate_at_k(recommended_list, bought_list, k=k)}')

Hit rate @1: 0
Hit rate @3: 0
Hit rate @5: 1
Hit rate @10: 1


#### Задание 2

In [5]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [6]:
def precision_at_k(recommended_list, bought_list, k):
    """calculating precision@k"""
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def money_precision_at_k_(recommended_list, bought_list, prices_recommended, k=5):
    
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [7]:
# посмотрим precision при разных k 
ks = [1, 3, 5, 10]
for k in ks:
    print(f'Precision @{k}: {precision_at_k(recommended_list, bought_list, k=k):.2f}')

Precision @1: 1.00
Precision @3: 0.33
Precision @5: 0.40
Precision @10: 0.20


#### Задание 3

In [8]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 32, 43] #id товаров
bought_list = [521, 32, 143, 991]

prices_recommended = [item*2.5 for item in recommended_list]
prices_bought = [item*2.5 for item in bought_list]

In [9]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k):

    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]

    flags = np.isin(bought_list, recommend_list) # что купили
    print('Реальная прибыль:', np.dot(flags, prices_bought).sum())
    print('Потенциальная прибыль:', prices_recommended.sum()) 
    
    # сколько купили / сколько могли купить, если бы все товары были релеванныт
    money_recall = np.dot(flags, prices_bought).sum() / prices_recommended.sum() 
    return money_recall

In [10]:
# посмотрим precision при разных k 
ks = [1, 3, 5, 10]
for k in ks:
    print('-------------------------------------------')
    print(f'Recall @{k}: {recall_at_k(recommended_list, bought_list, k=k):.2f}')
    print(f'Money recall @{k}:{money_recall_at_k(recommended_list, bought_list,prices_recommended, prices_bought, k=k):.2f}')
    
    
    

-------------------------------------------
Recall @1: 0.25
Реальная прибыль: 357.5
Потенциальная прибыль: 357.5
Money recall @1:1.00
-------------------------------------------
Recall @3: 0.25
Реальная прибыль: 357.5
Потенциальная прибыль: 3582.5
Money recall @3:0.10
-------------------------------------------
Recall @5: 0.50
Реальная прибыль: 2835.0
Потенциальная прибыль: 6127.5
Money recall @5:0.46
-------------------------------------------
Recall @10: 0.75
Реальная прибыль: 2915.0
Потенциальная прибыль: 19867.5
Money recall @10:0.15


#### Задание 4

In [11]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                           ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1, 2]] # юзер 3


In [12]:
def map_k(recommended_list, bought_list, k=5):
    U = len(bought_list_3_users) # кол-во юзеров
    mapk = 0

    for rec_list, bought_list in zip(recommended_list, bought_list_3_users):
        bought_list = np.array(bought_list)
        rec_list = np.array(rec_list)[:k]
        # индексы рекомендованного релевантного товара
        relevant_indexes = np.nonzero(np.isin(rec_list, bought_list))[0]
        
        # precision_at_k для каждого юзера
        sum_ = sum([precision_at_k(rec_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
        
        if len(relevant_indexes) == 0:
            ap_k = 0
        else:
            ap_k = sum_/len(relevant_indexes)
        mapk += ap_k

    return round(mapk/U, 2)


In [13]:
 map_k(recommended_list_3_users, bought_list_3_users, k=5)

0.33

#### Задание 5

In [14]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                           ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1, 2]] # юзер 3


In [15]:
def reciprocal_rank(recommended_list, bought_list, k=5):
    """calculating MRR - finding reciprocal rank 
            through the rank of first relevant recommendation for each user and taking its mean """
    n = len(bought_list_3_users) # кол-во юзеров
    rr_sum = 0

    for rec_list, bought_list in zip(recommended_list, bought_list_3_users):
        bought_list = np.array(bought_list)
        rec_list = np.array(rec_list)[:k]
        # индекс первого релевантного товара
        relevant_indexes = np.nonzero(np.isin(rec_list, bought_list))[0]
        
        # находим первое релевантное предсказание и вычисляем reciprocal rank для каждого юзера
        k_rank = relevant_indexes[True][0]
        if k_rank.size == 0:
            rr = 0
        else:
            rr = 1/(k_rank[0]+1)
            
        rr_sum +=rr

    return round(rr_sum/n, 2)

In [16]:
reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=5)

0.33